# Vox and PBS

### Author 
Stephen Lee

### Goal
Classify news source based on the article text. Training data: 
- Fox News
- PBS News

### Date 
First  : 4.8.19

Update : 6.24.19


## Read Data

In [1]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import GRU, Dense, Bidirectional, LSTM, Activation

import os 
import math 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn import metrics

Using TensorFlow backend.
/home/smlee_981/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
FOLDER_READ = '/home/smlee_981/data'
FILE = 'clean_article_df.csv'

In [3]:
os.getcwd()

'/home/smlee_981'

In [4]:
os.chdir(FOLDER_READ)

In [5]:
os.listdir()

['clean_article_df.csv', 'glove.840B.300d.txt']

In [6]:
df_all = pd.read_csv(FILE, sep='|').drop('Unnamed: 0', axis=1)
df_all.head()

article id source                                            article  \
0  fox_politics_166    Fox  <br>\nFormer New Jersey Gov. Chris Christie sa...   
1  fox_politics_390    Fox  FILE--In this July 28, 2016 file photo, Sen. B...   
2  fox_politics_423    Fox  Howard Kurtz: How Michael Cohen, Democrats sto...   
3  fox_politics_102    Fox  Student Union: Make UC Berkeley a sanctuary ca...   
4  fox_politics_492    Fox  President Trump’s health care executive order:...   

                                      clean_articles  targets  
0  <br>\nFormer New Jersey Gov. Chris Christie sa...        2  
1  FILE--In this July 28, 2016 file photo, Sen. B...        2  
2  Howard Kurtz: How Michael Cohen, Democrats sto...        2  
3  Student Union: Make UC Berkeley a sanctuary ca...        2  
4  President Trump’s health care executive order:...        2

# MAKE DATASETS

## First Dataset

#### Remove Fox

In [7]:
df_all = df_all[df_all['source'] != "Fox"].drop('article', axis=1)
df_all.groupby('source').count()

article id  clean_articles  targets
source                                     
PBS           1739            1739     1739
Vox           1027            1027     1027

In [9]:
pbs = df_all[df_all['source'] == 'PBS']
vox = df_all[df_all['source'] == 'Vox']

#### Duplicate Vox to balance

In [10]:
# balance df by resampling from vox 
diff = len(pbs) - len(vox)
df_first = df_all.append(vox.sample(diff, replace=True), ignore_index=True)
df_first.groupby('source').count()

article id  clean_articles  targets
source                                     
PBS           1739            1739     1739
Vox           1739            1739     1739

#### Relabel the targets

In [11]:
from tqdm import tqdm
import numpy as np

In [13]:
def label_fox(source):
    if source == "PBS":
        return 1 
    elif source == "Vox": 
        return 0
    else: 
        print(source)
        return None

targets = np.array([label_fox(t) for t in tqdm(df_first["source"])])
df_first['targets'] = targets

100%|██████████| 3478/3478 [00:00<00:00, 1239720.35it/s]


In [14]:
# make sure that the targets are correct
df_first.groupby('source').describe()

targets                                   
         count mean  std  min  25%  50%  75%  max
source                                           
PBS     1739.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Vox     1739.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

## Second Dataset
#### Sample from Vox to balance

In [15]:
vox.count()

article id        1027
source            1027
clean_articles    1027
targets           1027
dtype: int64

In [16]:
pbs.count()

article id        1739
source            1739
clean_articles    1739
targets           1739
dtype: int64

In [17]:
# make second dataset for training
# start with vox and append with a sample of PBS
df_second = vox.copy()

len_vox = len(vox)
df_second = df_second.append(pbs.sample(len_vox), ignore_index=True)
df_second.groupby('source').count()

article id  clean_articles  targets
source                                     
PBS           1027            1027     1027
Vox           1027            1027     1027

#### Relabel the targets

In [18]:
from tqdm import tqdm
import numpy as np

In [19]:
targets = np.array([label_fox(t) for t in tqdm(df_second["source"])])
df_second['targets'] = targets
df_second.head()

100%|██████████| 2054/2054 [00:00<00:00, 1094397.92it/s]


article id source  \
0   vox_politics_396    Vox   
1   vox_politics_372    Vox   
2   vox_politics_602    Vox   
3  vox_politics_1198    Vox   
4   vox_politics_682    Vox   

                                      clean_articles  targets  
0  Senate Republicans on Thursday revealed the Be...        0  
1  “New York will be destroyed,” the state’s Gov....        0  
2  The Trump administration wants to send a messa...        0  
3  Donald Trump’s long, improbable journey to pol...        0  
4  The Trump administration threw the fate of the...        0

In [21]:
# make sure that the targets are correct
df_second.groupby('source').describe()

targets                                   
         count mean  std  min  25%  50%  75%  max
source                                           
PBS     1027.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Vox     1027.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

## Get Embeddings and Define Helper Functions

In [22]:
# glove embeddings and data are in same folder

EMBEDS = 'glove.840B.300d.txt'

embeddings_index = {}
 
with open(EMBEDS, encoding='utf8') as embed:
    for line in tqdm(embed):
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print("Found {n} word vectors".format(n=len(embeddings_index)))

2196017it [03:49, 9555.75it/s]

Found 2196016 word vectors


In [23]:
def text_to_array(text, article_length=500):
    empty_emb = np.zeros(300)                   # each word is represented by a length 300 vector
    text = text[:-1].split()[:article_length]   # each article is length 500
    
    # look for word embedding, return zero array otherwise. 
    embeds = [embeddings_index.get(x, empty_emb) for x in text]
    embeds += [empty_emb] * (article_length - len(embeds))
    return np.array(embeds)

In [24]:
def batch_gen(train_df, batch_size=64, article_length=500):
    n = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.0)
        
        for i in range(n):
            texts = train_df['clean_articles'][i*batch_size: (i+1)*batch_size]
            targets = np.array(train_df['targets'][i*batch_size: (i+1)*batch_size])
            text_arr = np.array([text_to_array(text, article_length=article_length) for text in texts])
            yield text_arr, targets

# Train Dataset 1

### Define Models

#### Model 1: Bidirectional LSTM

In [25]:
# parameters
ARTICLE_LENGTH = 500
BATCH_SIZE = 64
DROPOUT = 0.2
REC_DROPOUT = 0.1

In [26]:
# SINGLE LAYER BIDIRECTIONAL LTSM
# 
# note...
#
#      batch_size         -> words per batch
#      article_length     -> words per article
#      embed_length       -> vector length per word

input_shape = (ARTICLE_LENGTH, 300)
lstm_in = int(BATCH_SIZE/2)

model_1 = Sequential()
model_1.add(Bidirectional(LSTM(lstm_in, return_sequences=False, \
                        dropout=DROPOUT, recurrent_dropout=REC_DROPOUT), \
                        input_shape=input_shape))

model_1.add(Activation('relu'))
#model.add(Bidirectional(LSTM(lstm_in)))

model_1.add(Dense(1, activation="sigmoid"))
model_1.compile(loss="binary_crossentropy", \
              optimizer="adam", \
              metrics=["accuracy"])

model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 64)                85248     
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 85,313
Trainable params: 85,313
Non-trainable params: 0
_________________________________________________________________


#### Model 2: Regular LSTM

In [30]:
# SINGLE LAYER BIDIRECTIONAL LTSM
# 
# note...
#
#      batch_size         -> words per batch
#      article_length     -> words per article
#      embed_length       -> vector length per word

input_shape = (ARTICLE_LENGTH, 300)
lstm_in = int(BATCH_SIZE)

model_2 = Sequential()
model_2.add(LSTM(lstm_in, return_sequences=False, dropout=DROPOUT, \
                 recurrent_dropout=REC_DROPOUT, input_shape=input_shape))

model_2.add(Activation('relu'))

model_2.add(Dense(1, activation="sigmoid"))
model_2.compile(loss="binary_crossentropy", \
              optimizer="adam", \
              metrics=["accuracy"])

model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 64)                93440     
_________________________________________________________________
activation_3 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 93,505
Trainable params: 93,505
Non-trainable params: 0
_________________________________________________________________


#### Split into test and training

In [31]:
train_df, test_df = train_test_split(df_first, test_size=0.1)

#### Check for similarity between test and training

In [32]:
test_df.groupby('source').describe()

targets                                   
         count mean  std  min  25%  50%  75%  max
source                                           
PBS      172.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Vox      176.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [33]:
train_df.groupby('source').describe()

targets                                   
         count mean  std  min  25%  50%  75%  max
source                                           
PBS     1567.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Vox     1563.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

#### Prepare test set for validation

In [34]:
# witheld for validation

x_test = np.array([text_to_array(x, article_length=ARTICLE_LENGTH) \
                          for x in tqdm(test_df["clean_articles"])])
y_test = np.array(test_df["targets"])

100%|██████████| 348/348 [00:00<00:00, 503.22it/s]


#### Train

In [35]:
data = batch_gen(train_df, batch_size=BATCH_SIZE, article_length=ARTICLE_LENGTH)
model_1.fit_generator(data, epochs=2, steps_per_epoch=250, \
                    validation_data=None, verbose=True)

Epoch 1/2
 98/250 [==========>...................] - ETA: 2:17 - loss: 0.4898 - acc: 0.7769

KeyboardInterrupt: 

In [24]:
data = batch_gen(train_df, batch_size=BATCH_SIZE, article_length=ARTICLE_LENGTH)
model_2.fit_generator(data, epochs=2, steps_per_epoch=250, \
                    validation_data=None, verbose=True)

Epoch 1/2
250/250 [==============================] - 187s 749ms/step - loss: 0.4004 - acc: 0.8211
Epoch 2/2
250/250 [==============================] - 181s 723ms/step - loss: 0.2704 - acc: 0.8880


#### Look at predictions

In [25]:
y_pred_1 = model_1.predict(x_test)
y_pred_1[:7]

array([[ 0.98302072],
       [ 0.22107595],
       [ 0.91881382],
       [ 0.51893014],
       [ 0.72391969],
       [ 0.88273358],
       [ 0.97644663]], dtype=float32)

In [25]:
y_pred_2 = model_2.predict(x_test)
y_pred_2[:7]

array([[ 0.98302072],
       [ 0.22107595],
       [ 0.91881382],
       [ 0.51893014],
       [ 0.72391969],
       [ 0.88273358],
       [ 0.97644663]], dtype=float32)

In [26]:
test_df[['source', 'clean_articles', 'targets']].head(7)

source                                     clean_articles  targets
1162    PBS    President Donald Trump’s son-in-law Jared Ku...        1
3010    Vox  The 2018 midterms are shaping up to be a histo...        0
1342    PBS  Former Trump campaign chairman Paul Manafort c...        1
2171    PBS    It’s a pivotal moment for Nancy Pelosi  the ...        1
513     Vox  Jim Acosta is getting his press pass back, aft...        0
1240    PBS  Democrats on the House intelligence committee ...        1
2431    PBS  President Donald Trump is assailing Democrats ...        1

In [27]:
for i in np.arange(0.25, 0.8, 0.05):
    res_1 = metrics.f1_score(y_test, y_pred_1 > i)
    res_2 = metrics.f1_score(y_test, y_pred_2 > i)
    print("Threshold {i} \nF1 score model 1: {res_1} \nF1 score model 2: {res_2} \n".format(\
                                                                                       i=round(i,2), \
                                                                                       res_1 = round(res_1, 3), \
                                                                                       res_2 = round(res_2, 3)))

Threshold 0.25, f1 score 0.8779220779220779
Threshold 0.3, f1 score 0.89247311827957
Threshold 0.35, f1 score 0.88268156424581
Threshold 0.4, f1 score 0.875
Threshold 0.45, f1 score 0.8786127167630058
Threshold 0.5, f1 score 0.8786127167630058
Threshold 0.55, f1 score 0.8705882352941177
Threshold 0.6, f1 score 0.8698224852071006
Threshold 0.65, f1 score 0.8648648648648648
Threshold 0.7, f1 score 0.8421052631578947
Threshold 0.75, f1 score 0.8375


# Train Dataset 2

### Refresh Models

#### Model 1: Bidirectional LSTM

In [21]:
# parameters
ARTICLE_LENGTH = 500
BATCH_SIZE = 64
DROPOUT = 0.2
REC_DROPOUT = 0.1

In [23]:
# SINGLE LAYER BIDIRECTIONAL LTSM
# 
# note...
#
#      batch_size         -> words per batch
#      article_length     -> words per article
#      embed_length       -> vector length per word

input_shape = (ARTICLE_LENGTH, 300)
lstm_in = int(BATCH_SIZE/2)

model_1 = Sequential()
model_1.add(Bidirectional(LSTM(lstm_in, return_sequences=False, \
                        dropout=DROPOUT, recurrent_dropout=REC_DROPOUT), \
                        input_shape=input_shape))

model_1.add(Activation('relu'))
#model.add(Bidirectional(LSTM(lstm_in)))

model_1.add(Dense(1, activation="sigmoid"))
model_1.compile(loss="binary_crossentropy", \
              optimizer="adam", \
              metrics=["accuracy"])

model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 64)                85248     
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 85,313
Trainable params: 85,313
Non-trainable params: 0
_________________________________________________________________


#### Model 2: Regular LSTM

In [ ]:
# SINGLE LAYER BIDIRECTIONAL LTSM
# 
# note...
#
#      batch_size         -> words per batch
#      article_length     -> words per article
#      embed_length       -> vector length per word

input_shape = (ARTICLE_LENGTH, 300)
lstm_in = int(BATCH_SIZE)

model_2 = Sequential()
model_2.add(LSTM(lstm_in, return_sequences=False, \
                        dropout=DROPOUT, recurrent_dropout=REC_DROPOUT), \
                        input_shape=input_shape)

model_2.add(Activation('relu'))

model_2.add(Dense(1, activation="sigmoid"))
model_2.compile(loss="binary_crossentropy", \
              optimizer="adam", \
              metrics=["accuracy"])

model_2.summary()

#### Split into test and training

In [34]:
train_df, test_df = train_test_split(df_second, test_size=0.1)
train_df.groupby('source').describe()

targets                                   
         count mean  std  min  25%  50%  75%  max
source                                           
PBS      928.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Vox      920.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

#### Check for similarity between test and training

In [35]:
test_df.groupby('source').describe()

targets                                   
         count mean  std  min  25%  50%  75%  max
source                                           
PBS       99.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Vox      107.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [36]:
train_df.groupby('source').describe()

targets                                   
         count mean  std  min  25%  50%  75%  max
source                                           
PBS      928.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
Vox      920.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

#### prepare test set for validation

In [38]:
# witheld for validation
 
x_test = np.array([text_to_array(x, article_length=ARTICLE_LENGTH) \
                          for x in tqdm(test_df["clean_articles"])])
y_test = np.array(test_df["targets"])

100%|██████████| 206/206 [00:00<00:00, 710.73it/s]


#### train

In [40]:
data = batch_gen(train_df, batch_size=BATCH_SIZE, article_length=ARTICLE_LENGTH)
model_1.fit_generator(data, epochs=2, steps_per_epoch=250, \
                    validation_data=None, verbose=True)

Epoch 1/2
250/250 [==============================] - 184s 737ms/step - loss: 0.3917 - acc: 0.8285
Epoch 2/2
250/250 [==============================] - 180s 720ms/step - loss: 0.2566 - acc: 0.8944


In [40]:
data = batch_gen(train_df, batch_size=BATCH_SIZE, article_length=ARTICLE_LENGTH)
model_2.fit_generator(data, epochs=2, steps_per_epoch=250, \
                    validation_data=None, verbose=True)

Epoch 1/2
250/250 [==============================] - 184s 737ms/step - loss: 0.3917 - acc: 0.8285
Epoch 2/2
250/250 [==============================] - 180s 720ms/step - loss: 0.2566 - acc: 0.8944


#### Look at predictions

In [41]:
y_pred_1 = model_1.predict(x_test)
y_pred_1[:7]

/home/smlee_981/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


article id source  \
702    vox_politics_711    Vox   
1266   pbs_politics_183    PBS   
1068  pbs_politics_1296    PBS   
829    vox_politics_980    Vox   
1528   pbs_politics_398    PBS   

                                         clean_articles  targets  pred_raw  
702   The Trump administration has decided to cut of...        0  0.004916  
1266  OAKLAND, Calif.  Democratic Sen. Kamala Harris...        1  0.895678  
1068    Christine Blasey Ford may testify against Su...        1  0.976337  
829   The 2018 midterms are shaping up to be a histo...        0  0.162000  
1528  President Donald Trump is defending the hush m...        1  0.980907

In [41]:
y_pred_2 = model_2.predict(x_test)
y_pred_2[:7]

/home/smlee_981/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


article id source  \
702    vox_politics_711    Vox   
1266   pbs_politics_183    PBS   
1068  pbs_politics_1296    PBS   
829    vox_politics_980    Vox   
1528   pbs_politics_398    PBS   

                                         clean_articles  targets  pred_raw  
702   The Trump administration has decided to cut of...        0  0.004916  
1266  OAKLAND, Calif.  Democratic Sen. Kamala Harris...        1  0.895678  
1068    Christine Blasey Ford may testify against Su...        1  0.976337  
829   The 2018 midterms are shaping up to be a histo...        0  0.162000  
1528  President Donald Trump is defending the hush m...        1  0.980907

In [42]:
for i in np.arange(0.25, 0.8, 0.05):
    res_1 = metrics.f1_score(y_test, y_pred_1 > i)
    res_2 = metrics.f1_score(y_test, y_pred_2 > i)
    print("Threshold {i} \nF1 score model 1: {res_1} \nF1 score model 2: {res_2} \n".format(\
                                                                                       i=round(i,2), \
                                                                                       res_1 = round(res_1, 3), \
                                                                                       res_2 = round(res_2, 3)))

Threshold 0.25, f1 score 0.8518518518518517
Threshold 0.3, f1 score 0.8679245283018867
Threshold 0.35, f1 score 0.8666666666666667
Threshold 0.4, f1 score 0.8585365853658536
Threshold 0.45, f1 score 0.864321608040201
Threshold 0.5, f1 score 0.8730964467005075
Threshold 0.55, f1 score 0.865979381443299
Threshold 0.6, f1 score 0.869109947643979
Threshold 0.65, f1 score 0.8736842105263157
Threshold 0.7, f1 score 0.8617021276595745
Threshold 0.75, f1 score 0.8524590163934426
